<a href="https://colab.research.google.com/github/Rachit180/FER/blob/main/FER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import keras
from matplotlib import pyplot
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras import regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam, RMSprop, SGD, Adamax,Nadam
from keras.preprocessing.image import ImageDataGenerator,load_img
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization, MaxPooling2D, Activation, Input, ZeroPadding2D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.regularizers import l1, l2
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler

from tensorflow.keras.models import load_model

ImportError: cannot import name 'ImageDataGenerator' from 'keras.preprocessing.image' (/usr/local/lib/python3.10/dist-packages/keras/api/preprocessing/image/__init__.py)

In [ ]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from keras.layers import Conv2D, MaxPool2D, AveragePooling2D, Input, BatchNormalization, MaxPooling2D, Activation, Flatten, Dense, Dropout
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler
from keras.preprocessing import image
import scipy
import os
import cv2

In [ ]:
data = pd.read_csv('/content/fer2013.csv')
path='/content/fer2013.csv'

In [ ]:
data.head(5)

In [ ]:
# Split the pixel values into lists
train_pixels = data['pixels'].str.split(" ").tolist()

# Find the maximum length of lists
max_length = max(len(pixels) for pixels in train_pixels)

# Pad the lists with zeros to make them of equal length
train_pixels = [pixels + ['0'] * (max_length - len(pixels)) for pixels in train_pixels]

# Convert the list of lists to numpy array of uint8
train_pixels = np.uint8(train_pixels)


In [ ]:
import numpy as np

# Iterate over the dataset to extract pixel values
train_pixels = []
max_length = 0  # Initialize maximum length to 0
for index, row in data.iterrows():
    pixels = row['pixels'].split(" ")
    pixels = [int(pixel) for pixel in pixels]
    train_pixels.append(pixels)
    if len(pixels) > max_length:
        max_length = len(pixels)

# Pad the lists with zeros to make them of equal length
for pixels in train_pixels:
    pixels.extend([0] * (max_length - len(pixels)))

# Convert the list of lists to numpy array
train_pixels = np.uint8(train_pixels)


In [ ]:
def create(num):
  image = train_pixels[num]
  grid = image.reshape((48, 48))
  plt.imshow(grid, cmap='viridis')
  plt.title(data['emotion'][num])

In [ ]:
create(33)

In [ ]:
x_data = data['pixels']
y_data = data['emotion']

y_data.value_counts()

In [ ]:
from imblearn.over_sampling import SMOTE

oversampler = RandomOverSampler(sampling_strategy='all')

x_data, y_data = oversampler.fit_resample(x_data.values.reshape(-1,1), y_data)
print(x_data.shape," ",y_data.shape)

In [ ]:
x_data = pd.Series(x_data.flatten())
x_data

In [ ]:
x_data = np.array(list(map(str.split, x_data)), np.float32)
x_data /= 255
print(x_data[:10])

In [ ]:
x_data = x_data.reshape(-1, 48, 48, 1)
x_data.shape

In [ ]:
y_data = np.array(y_data)
print(y_data)
y_data = y_data.reshape(y_data.shape[0], 1)
y_data.shape
print(y_data)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense,GlobalAveragePooling2D

model = Sequential()

# Input Layer
model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(48, 48, 1), activation='relu'))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(GlobalAveragePooling2D())
model.add(Dense(200, activation='relu'))
model.add(Dense(7, activation='softmax'))

model.summary()

model.compile(
    optimizer=Nadam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
from tensorflow.keras import layers
def double_conv_block(x, n_filters):
   # Conv2D then ReLU activation
    x = layers.Conv2D(n_filters, 3, padding = "same", activation = "relu", kernel_initializer = "he_normal")(x)
#    # Conv2D then ReLU activation
#     x = layers.Conv2D(n_filters, 3, padding = "same", activation = "relu", kernel_initializer = "he_normal")(x)
    return x
def downsample_block(x, n_filters):
    f = double_conv_block(x, n_filters)
    p = layers.MaxPool2D(2)(f)
    p = layers.Dropout(0.3)(p)
    return f, p
def upsample_block(x, conv_features, n_filters):
   # upsample
    x = layers.Conv2DTranspose(n_filters, 3, 2, padding="same")(x)
   # concatenate
    x = layers.concatenate([x, conv_features])
   # dropout
    x = layers.Dropout(0.3)(x)
   # Conv2D twice with ReLU activation
    x = double_conv_block(x, n_filters)
    return x

def build_unet_model():
    inputs = layers.Input(shape=(48,48,1))
   # encoder: contracting path - downsample
   # 1 - downsample
    f1, p1 = downsample_block(inputs, 8)
   # 2 - downsample
    f2, p2 = downsample_block(p1, 16)
   # 3 - downsample
    f3, p3 = downsample_block(p2, 32)
   # 4 - downsample

#     f4, p4 = downsample_block(p3, 64)

   # 5 - bottleneck
    bottleneck = double_conv_block(p3, 64)
   # decoder: expanding path - upsample
   # 6 - upsample
#     u6 = upsample_block(bottleneck, f4, 64)
   # 7 - upsample
    u7 = upsample_block(bottleneck, f3, 32)
   # 8 - upsample
    u8 = upsample_block(u7, f2, 16)
   # 9 - upsample
    u9 = upsample_block(u8, f1, 8)

   # outputs
    gap = layers.GlobalAveragePooling2D()(u9)

#     o = layers.Flatten()(u9)
#     o1 = layers.Dense(10, activation = "relu")(o)
    outputs = layers.Dense(7, activation = "softmax")(gap)
#     outouts=layers.Flatten()
   # unet model with Keras Functional API
    unet_model = tf.keras.Model(inputs, outputs, name="U-Net")

    return unet_model

In [ ]:
from tensorflow.keras import layers

def conv_block(x, n_filters, strides=(1, 1)):
    shortcut = x

    x = layers.Conv2D(n_filters, 3, strides=strides, padding='same', activation=None, kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(n_filters, 3, strides=(1, 1), padding='same', activation=None, kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)

    if strides != (1, 1) or shortcut.shape[-1] != n_filters:
        shortcut = layers.Conv2D(n_filters, 1, strides=strides, padding='valid', activation=None, kernel_initializer='he_normal')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

    x = layers.Add()([x, shortcut])
    x = layers.Activation('relu')(x)

    return x

def build_resnet_model():
    inputs = layers.Input(shape=(48, 48, 1))

    x = layers.Conv2D(64, 7, strides=(2, 2), padding='same', activation=None, kernel_initializer='he_normal')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

    # Stage 1
    x = conv_block(x, 64, strides=(1, 1))
    x = conv_block(x, 64, strides=(1, 1))

    # Stage 2
    x = conv_block(x, 128, strides=(2, 2))
    x = conv_block(x, 128, strides=(1, 1))

    # Stage 3
    x = conv_block(x, 256, strides=(2, 2))
    x = conv_block(x, 256, strides=(1, 1))

    # Stage 4
    x = conv_block(x, 512, strides=(2, 2))
    x = conv_block(x, 512, strides=(1, 1))

    # Global Average Pooling
    x = layers.GlobalAveragePooling2D()(x)

    # Output
    outputs = layers.Dense(7, activation='softmax')(x)

    # ResNet model with Keras Functional API
    resnet_model = tf.keras.Model(inputs, outputs, name='ResNet')

    return resnet_model


In [ ]:
resnet_model = build_resnet_model()
resnet_model.summary()

In [ ]:
unet_model = build_unet_model()
unet_model.summary()

In [ ]:
tf.keras.utils.plot_model(unet_model, show_shapes=True)

In [ ]:
nag = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
unet_model.compile(optimizer=Nadam(learning_rate=0.005), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
print(y_train.shape)
y_train = y_train.reshape((-1,1))
print(y_train.shape)
y_test = y_test.reshape((-1,1))

In [ ]:
print(y_train.shape)
y_train = to_categorical(y_train, 7)

print(y_train.shape)

In [ ]:
y_test = to_categorical(y_test, 7)
y_test.shape

In [ ]:
y_val = to_categorical(y_val, 7)
y_val.shape

In [ ]:
checkpointer = [EarlyStopping(monitor = 'val_accuracy', verbose = 1,
                              restore_best_weights=True,mode="max",patience = 10),
                ModelCheckpoint('best_model.h5',monitor="val_accuracy",verbose=1,
                                save_best_only=True,mode="max")]

In [ ]:
history = unet_model.fit(X_train, y_train,
                    epochs=60,
                    batch_size=64,
                    verbose=1,
                    callbacks=[checkpointer],
                    validation_data=(X_val, y_val))

In [ ]:
history = model.fit(X_train, y_train,
                    epochs=60,
                    batch_size=64,
                    verbose=1,
                    callbacks=[checkpointer],
                    validation_data=(X_val, y_val))

In [ ]:
print("Accuracy of our model on test data : " , model.evaluate(X_test,y_test)[1]*100 , "%")

In [ ]:
model.save('fer_2013_74.60_resnet.keras')